In [7]:
import csv
import pandas as pd
from pandas.api.types import is_period_dtype

import asyncio
from typing import List, Dict, Optional
from playwright.async_api import async_playwright, Page
from urllib.parse import urljoin
from datetime import datetime
import os
import math
import time
import datetime as dt
from typing import Optional, Tuple
import requests
import duckdb

import asyncio, csv, re
from datetime import datetime
from urllib.parse import urljoin
from playwright.async_api import async_playwright, Page
from datetime import datetime, timezone
from dotenv import load_dotenv
load_dotenv()

True

# 🧠 Projeto: Enriquecimento de Base Analítica com Web Scraping e API Financeira

## 🎯 Contexto

Uma **fintech de investimentos** precisa enriquecer sua base analítica com informações externas do mercado para apoiar decisões estratégicas.
Como Engenheira de Dados, foi desenvolvido um **pipeline de dados** que coleta informações públicas de **notícias** e **séries financeiras**, armazena localmente em um **banco DuckDB**, e permite posterior exploração via SQL e dashboards.

---

## 🧩 Objetivo

Construir um pipeline completo de **coleta, transformação e carga (ETL)** que una:

* **Web Scraping** de notícias econômicas e geopolíticas (BBC News);
* **API Pública** de dados financeiros (FRED e CoinGecko);
* **Integração analítica** em banco local **DuckDB**.

---

## ⚙️ Stack Utilizada

| Etapa         | Tecnologia                        | Descrição                                          |
| ------------- | --------------------------------- | -------------------------------------------------- |
| Coleta Web    | `Playwright` + `asyncio`          | Scraping assíncrono de páginas de notícias da BBC  |
| Coleta API    | `requests`, `pandas`              | Consumo de APIs FRED (Federal Reserve) e CoinGecko |
| Armazenamento | `DuckDB`                          | Banco analítico local com três tabelas             |
| Ambiente      | `Python 3.9+`, `Jupyter Notebook` | Execução e análise                                 |
| Persistência  | `.duckdb`, `.parquet`, `.csv`     | Formatos intermediários                            |

---

## 🌐 Fontes de Dados

### 🔹 Notícias (Web Scraping – BBC News)

* Fonte: [BBC News – US-Canada](https://www.bbc.com/news/us-canada)
* Coletadas **100 notícias** contendo título, resumo, link e data de coleta.
* Campos armazenados:

  ```
  ['title', 'url', 'summary', 'collected_at']
  ```
* Objetivo: capturar contexto geopolítico e eventos com impacto em mercados.

### 🔹 Séries Financeiras (APIs Públicas)

| Fonte     | Série          | Descrição                                   | Período  |
| --------- | -------------- | ------------------------------------------- | -------- |
| FRED      | `DCOILBRENTEU` | Preço diário do petróleo Brent (USD/barril) | 6+ meses |
| FRED      | `DEXUSUK`      | Taxa USD/GBP (invertida para GBP/USD)       | 6+ meses |
| CoinGecko | `BTC/USD`      | Cotação diária do Bitcoin                   | 6+ meses |

Os dados foram padronizados em base diária contínua, com cálculo de retornos em janelas de 1, 3 e 5 dias (`r1`, `r3`, `r5`).

---

## 🗄️ Modelagem de Dados no DuckDB

### Tabelas criadas:

| Tabela          | Descrição                       | Principais Campos                          |
| --------------- | ------------------------------- | ------------------------------------------ |
| **prices**      | Séries históricas dos ativos    | `instr`, `date`, `close`, `r1`, `r3`, `r5` |
| **news_bbc**    | Notícias coletadas via scraping | `title`, `url`, `summary`, `collected_at`, `published_at`, `published_text`  |
| **instruments** | Metadados dos instrumentos      | `instr_id`, `symbol`, `name`, `class`      |

```sql
-- Exemplo de schema no DuckDB
DESCRIBE prices;
DESCRIBE news_bbc;
DESCRIBE instruments;
```

---

## 📊 Resultados

* **100 notícias** coletadas da BBC News.
* **3 instrumentos** (Brent, GBP/USD, BTC/USD) com **211 dias** de dados cada.
* **3 tabelas analíticas** armazenadas no DuckDB (`prices`, `news_bbc`, `instruments`).
* Pipeline totalmente reprodutível e modular, pronto para expansão com novos tópicos ou ativos.

---

## 🧾 Estrutura Final

```
📂 projeto_etl_fintech/
│
├── market.duckdb                 # Banco analítico local
├── prices.parquet                # Dados de preços
├── bbc_us_canada_latest_updates.csv       # Notícias coletadas
├── projeto_final_web_scraping.ipynb            # Notebook principal
└── requirements.txt              # Dependências fixas
```

---

## ✅ Conclusão

O projeto integra dados não estruturados (notícias) e estruturados (séries econômicas), simulando um fluxo real de engenharia de dados.
Com as tabelas organizadas no DuckDB, é possível executar consultas SQL rápidas e realizar análises temporais sobre o impacto de eventos geopolíticos nos ativos financeiros.

## Configs

In [2]:
DUCKDB_PATH = os.getenv("DUCKDB_PATH")

In [3]:
# =========================
# Configs
# =========================
# 100 últimas notícias do BBC US & Canada

START_URL = os.getenv("NEWS_SOURCE")
TARGET = 100
BASE = "https://www.bbc.com"
# varredura começa por várias seções de /news (aumenta cobertura)
START_URLS = [
    "https://www.bbc.com/news",
    "https://www.bbc.com/news/world",
    "https://www.bbc.com/news/uk",
    "https://www.bbc.com/news/business",
    "https://www.bbc.com/news/technology",
    "https://www.bbc.com/news/science_and_environment",
    "https://www.bbc.com/news/entertainment_and_arts",
]
OUTCSV = "bbc_us_canada_latest_updates.csv"

# Somente artigos válidos:
ART_RE = re.compile(r"^https?://(?:www\.)?bbc\.com/news/articles/[A-Za-z0-9]+(?:[/?#]|$)")

In [4]:
# =========================
# Configs
# =========================
# Período alvo (>= 6 meses); pego ~210 dias\

END = dt.date.today()
START = END - dt.timedelta(days=210)  # ~7 meses

FRED_API_KEY = os.getenv("FRED_API_KEY")  # .env
FRED_BASE = "https://api.stlouisfed.org/fred/series/observations"
FRED_SERIES = {
    "BRENT": "DCOILBRENTEU",  # Brent Europe, diário
    "GBPUSD": "DEXUSUK",      # Taxa USD/GBP diária 
}

COINGECKO_BASE = "https://api.coingecko.com/api/v3"
COINGECKO_COIN = "bitcoin"
COINGECKO_VS = "usd"

## Web scraping de notícias 

In [ ]:
# Um "listing" (página de lista) precisa estar em /news e NÃO ser um artigo:
def is_news_listing_url(url: str) -> bool:
    try:
        u = urlparse(url)
        if u.netloc not in ("www.bbc.com", "bbc.com"): return False
        if not u.path.startswith("/news"): return False
        if "/news/articles/" in u.path:  # artigo (não é listing)
            return False
        # evita áreas sabidamente fora do news feed tradicional
        if any(seg in u.path for seg in ("/reel/", "/future/", "/innovation/", "/culture/")):
            return False
        return True
    except:
        return False

# ---------- Utils ----------
def abs_url(href): 
    return urljoin(BASE, href or "")

def _now_iso():
    return datetime.now(timezone.utc).astimezone().isoformat()

def _norm_iso(dt: str) -> str:
    if not dt: return ""
    try:
        return datetime.fromisoformat(dt.replace("Z","+00:00")).astimezone(timezone.utc).isoformat()
    except Exception:
        return dt.strip()

# ---------- Cookies ----------
async def accept_cookies(page: Page):
    for sel in (
        '[data-testid="cookie-banner"] button:has-text("Accept")',
        'button:has-text("I Agree")',
        'button:has-text("Agree")',
        '#bbccookies-continue-button',
    ):
        b = page.locator(sel).first
        if await b.count() and await b.is_visible():
            await b.click(); break

# ---------- EXTRAÇÃO: links de artigos (news/articles/ID) ----------
async def extract_article_urls_on_page(page: Page):
    js = r"""
    () => {
      const BASE = 'https://www.bbc.com';
      const root = document.querySelector('main') || document;
      const anchors = Array.from(root.querySelectorAll('a[href*="/news/articles/"]'));
      const seen = new Set(), urls = [];
      for (const a of anchors) {
        const href = a.getAttribute('href') || '';
        try {
          const url = new URL(href, BASE).toString();
          if (!seen.has(url)) { seen.add(url); urls.push(url); }
        } catch {}
      }
      return urls;
    }
    """
    raw = await page.evaluate(js)
    urls = [u for u in raw if ART_RE.search(u)]
    print(f"[{page.url}] artigos válidos nesta página: {len(urls)}")
    return urls

# ---------- DESCOBRIR novas páginas de listagem (paginadores + subseções) ----------
async def discover_listing_pages(page: Page):
    """
    Encontra:
      - links que sugerem paginação (?page=, /page/N/, /page-N) dentro de /news
      - links de subseção ainda em /news (evitando reel/future/innovation/culture)
    """
    js = r"""
    () => {
      const BASE = 'https://www.bbc.com';
      const anchors = Array.from(document.querySelectorAll('a[href]'));
      const urls = [];
      for (const a of anchors) {
        const href = a.getAttribute('href') || '';
        try {
          const u = new URL(href, BASE).toString();
          urls.push(u);
        } catch {}
      }
      return urls;
    }
    """
    all_urls = await page.evaluate(js)
    out = set()
    for u in all_urls:
        if not is_news_listing_url(u): 
            continue
        # heurística: priorizar sinais de paginação OU subseções de /news
        if re.search(r"[?&]page=\d+", u) or re.search(r"/page/\d+/?", u) or re.search(r"/page-\d+/?", u):
            out.add(u)
        else:
            # subseções /news/... (sem page) também ajudam a diversificar
            out.add(u)
    # limita o volume por página para evitar explosão de crawling
    # prioriza paginadores explícitos primeiro
    pagers = [u for u in out if re.search(r"(?:[?&]page=\d+|/page/\d+|/page-\d+)", u)]
    subsecs = [u for u in out if u not in pagers]
    # devolve até 20 candidatos, pagers primeiro
    ordered = pagers[:15] + subsecs[:5]
    print(f"[{page.url}] novos listings candidatos: {len(ordered)}")
    return ordered

# ---------- ENRIQUECIMENTO POR REQUEST ----------
_JSONLD_RE = re.compile(
    r'<script[^>]+type=["\']application/ld\+json["\'][^>]*>(.*?)</script>',
    re.I | re.S
)

def _extract_meta(html: str, name: str, attr="content"):
    pat = rf'<meta[^>]+(?:property|name)=["\']{re.escape(name)}["\'][^>]*{attr}=["\']([^"\']+)["\']'
    m = re.search(pat, html, re.I)
    return m.group(1).strip() if m else ""

def _extract_tag_text(html: str, tag: str):
    m = re.search(rf'<{tag}[^>]*>(.*?)</{tag}>', html, re.I | re.S)
    if not m: return ""
    txt = re.sub(r"<[^>]+>", " ", m.group(1))
    return re.sub(r"\s+", " ", txt).strip()

def _first_nonempty(*vals):
    for v in vals:
        if v and str(v).strip():
            return str(v).strip()
    return ""

def _from_jsonld_block(obj):
    t = str(obj.get("@type"))
    if not any(k in t for k in ("Article","NewsArticle","LiveBlogPosting")):
        return ("","","")
    title = _first_nonempty(obj.get("headline"), obj.get("name"))
    summary = _first_nonempty(obj.get("description"), obj.get("alternativeHeadline"))
    pub = _first_nonempty(obj.get("datePublished"), obj.get("dateModified"))
    return (title, summary, pub)

def _parse_article_html(html: str):
    title = ""
    summary = ""
    published_iso = ""
    published_text = ""

    # 1) JSON-LD
    for m in _JSONLD_RE.finditer(html):
        try:
            data = json.loads(m.group(1))
            arr = data if isinstance(data, list) else [data]
            for d in arr:
                t, s, p = _from_jsonld_block(d)
                if t and not title: title = t
                if s and not summary: summary = s
                if p and not published_iso: published_iso = p
        except:
            pass

    # 2) Fallbacks
    if not title:
        title = _first_nonempty(
            _extract_meta(html, "og:title"),
            _extract_meta(html, "twitter:title"),
            _extract_tag_text(html, "title")
        )
    if not summary:
        summary = _first_nonempty(
            _extract_meta(html, "og:description"),
            _extract_meta(html, "twitter:description"),
            _extract_meta(html, "description", attr="content")
        )

    # 3) Datas
    m_dt = re.search(r'<time[^>]+datetime=["\']([^"\']+)["\']', html, re.I)
    if m_dt and not published_iso:
        published_iso = m_dt.group(1)

    m_txt = re.search(r'<time[^>]*>(.*?)</time>', html, re.I | re.S)
    if m_txt:
        published_text = re.sub(r"<[^>]+>", " ", m_txt.group(1))
        published_text = re.sub(r"\s+", " ", published_text).strip()

    if not published_iso:
        meta_pub = _extract_meta(html, "article:published_time")
        if meta_pub:
            published_iso = meta_pub

    return {
        "title": title.strip(),
        "summary": summary.strip(),
        "published_at": _norm_iso(published_iso),
        "published_text": published_text.strip()
    }

async def fetch_article_details(context, url: str) -> dict:
    try:
        resp = await context.request.get(url, timeout=30_000)
        if not resp.ok:
            return {"url": url, "title":"", "summary":"", "published_at":"", "published_text":""}
        html = await resp.text()
        data = _parse_article_html(html)
        data["url"] = url
        data["collected_at"] = _now_iso()
        return data
    except Exception:
        return {"url": url, "title":"", "summary":"", "published_at":"", "published_text":"", "collected_at":_now_iso()}

# ---------- SALVAR CSV ----------
async def save_csv(rows, path=OUTCSV):
    cols = ["title","summary","url","published_text","published_at","collected_at"]
    with open(path,"w",newline="",encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=cols)
        w.writeheader()
        for r in rows:
            w.writerow({k:r.get(k,"") for k in cols})
    print(f"✅ CSV salvo: {path}")

# ---------- CRAWLER PRINCIPAL ----------
async def crawl_news_articles(target: int = 100, max_visits: int = 120):
    """
    Busca pelo menos 'target' artigos news/articles/<ID> navegando apenas por listagens /news.
    Limita total de visitas (max_visits) pra evitar loop infinito.
    """
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()

        queue = list(START_URLS)  # começa por várias seções
        visited = set()
        found_articles = []
        seen_articles = set()

        while queue and len(found_articles) < target and len(visited) < max_visits:
            url = queue.pop(0)
            if url in visited: 
                continue
            visited.add(url)

            try:
                await page.goto(url, timeout=60_000, wait_until="domcontentloaded")
                await accept_cookies(page)
            except:
                continue

            # 1) pega artigos nesta página
            art_urls = await extract_article_urls_on_page(page)
            for u in art_urls:
                if u not in seen_articles:
                    seen_articles.add(u)
                    found_articles.append({"url": u, "collected_at": _now_iso()})
                    if len(found_articles) >= target:
                        break
            if len(found_articles) >= target:
                break

            # 2) descobre novas listagens para visitar (paginadores + subseções /news)
            candidates = await discover_listing_pages(page)
            for c in candidates:
                if c not in visited and is_news_listing_url(c):
                    queue.append(c)

        # ENRIQUECIMENTO paralelizado
        rows = found_articles[:target]
        sem = asyncio.Semaphore(10)
        async def worker(r):
            async with sem:
                info = await fetch_article_details(context, r["url"])
                r.update(info)
        await asyncio.gather(*(worker(r) for r in rows))

        await browser.close()
        return rows

# ---------- WRAPPER ----------
async def scrape_latest_updates(*args, **kwargs):
    """
    Compat:
      - scrape_latest_updates(100)
      - scrape_latest_updates(target=100)
    """
    target = kwargs.get("target", 40)
    if args and isinstance(args[0], int):
        target = args[0]
    return await crawl_news_articles(target=target, max_visits=130)

In [8]:
# =========================
# Execução
# =========================
data = await scrape_latest_updates(100)
await save_csv(data, OUTCSV)

[https://www.bbc.com/news] artigos válidos nesta página: 29
[https://www.bbc.com/news] novos listings candidatos: 0
[https://www.bbc.com/news/world] artigos válidos nesta página: 19
[https://www.bbc.com/news/world] novos listings candidatos: 0
[https://www.bbc.com/news/uk] artigos válidos nesta página: 27
[https://www.bbc.com/news/uk] novos listings candidatos: 0
[https://www.bbc.com/business] artigos válidos nesta página: 22
[https://www.bbc.com/business] novos listings candidatos: 0
[https://www.bbc.com/innovation] artigos válidos nesta página: 16
[https://www.bbc.com/innovation] novos listings candidatos: 0
[https://www.bbc.com/news/science_and_environment] artigos válidos nesta página: 57
✅ CSV salvo: bbc_us_canada_latest_updates.csv


In [22]:
news = pd.DataFrame(data)

In [23]:
news.head()

,url,collected_at,title,summary,published_at,published_text
0,https://www.bbc.com/news/articles/c2emmdnw82yo,2025-11-01T02:03:57.512675-03:00,Andrew Mountbatten Windsor will not leave Roya...,There are also no plans to formally remove And...,2025-10-31T20:22:09.991000+00:00,9 hours ago
1,https://www.bbc.com/news/articles/cy8vrzpgxnro,2025-11-01T02:03:57.575413-03:00,"Andrew: Why Sarah Ferguson, Beatrice and Eugen...",The scandal engulfing Andrew is of his own mak...,2025-10-31T19:21:48.330000+00:00,10 hours ago
2,https://www.bbc.com/news/articles/cn09r01k9yqo,2025-11-01T02:03:58.789745-03:00,What Justin Trudeau&#x27;s new era with Katy P...,Both the former Canadian prime minister and th...,2025-11-01T01:25:14.868000+00:00,4 hours ago
3,https://www.bbc.com/news/articles/ckg4q403rpzo,2025-11-01T02:03:58.353402-03:00,"Egypt&#x27;s Grand Museum opens, displaying Tu...",The launch of the billion-dollar site sees fre...,2025-11-01T01:19:28.608000+00:00,4 hours ago
4,https://www.bbc.com/news/articles/c0jdd186l0go,2025-11-01T02:03:57.554134-03:00,"Desperation in Black River, Jamaica, after Hur...",People in Black River haven&#x27;t seen any ai...,2025-11-01T02:10:13.047000+00:00,3 hours ago


In [24]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   url             100 non-null    object
 1   collected_at    100 non-null    object
 2   title           100 non-null    object
 3   summary         100 non-null    object
 4   published_at    100 non-null    object
 5   published_text  100 non-null    object
dtypes: object(6)
memory usage: 4.8+ KB


### Persistência

In [27]:
# --- 1) manter só as colunas com valor ---
news["collected_at"] = pd.to_datetime(news["collected_at"], utc=True, errors="coerce")
news["published_at"] = pd.to_datetime(news["published_at"], utc=True, errors="coerce")

news = news.drop_duplicates(subset=["url"]).reset_index(drop=True)

# --- 2) conectar ao banco local ---
con = duckdb.connect(DUCKDB_PATH)

# --- 3) criar a tabela de notícias com apenas as colunas ---
con.execute("""
DROP TABLE IF EXISTS news_bbc;
CREATE TABLE IF NOT EXISTS news_bbc (
    url           VARCHAR,
    collected_at  TIMESTAMP,
    title         VARCHAR,
    summary       VARCHAR,
    published_at TIMESTAMP,
    published_text VARCHAR
);
""")

# --- 4) inserir os dados ---
con.register("tmp_news", news)
con.execute("""
INSERT INTO news_bbc
SELECT url, 
        collected_at, 
        title, 
        summary, 
        published_at,
       published_text
FROM tmp_news;
""")

### Verificação

100 notícias extraídas no período de 01/08 até 01/11

In [30]:
# --- 5) checar resultado ---
con = duckdb.connect(DUCKDB_PATH)
print(con.execute("SELECT COUNT(*) AS n FROM news_bbc").df())
print(con.execute("SELECT * FROM news_bbc LIMIT 5").df())
print(con.execute("SELECT min(published_at), max(published_at) FROM news_bbc").df())
con.close()

     n
0  100
                                              url               collected_at  \
0  https://www.bbc.com/news/articles/c2emmdnw82yo 2025-11-01 02:03:57.512675   
1  https://www.bbc.com/news/articles/cy8vrzpgxnro 2025-11-01 02:03:57.575413   
2  https://www.bbc.com/news/articles/cn09r01k9yqo 2025-11-01 02:03:58.789745   
3  https://www.bbc.com/news/articles/ckg4q403rpzo 2025-11-01 02:03:58.353402   
4  https://www.bbc.com/news/articles/c0jdd186l0go 2025-11-01 02:03:57.554134   

                                               title  \
0  Andrew Mountbatten Windsor will not leave Roya...   
1  Andrew: Why Sarah Ferguson, Beatrice and Eugen...   
2  What Justin Trudeau&#x27;s new era with Katy P...   
3  Egypt&#x27;s Grand Museum opens, displaying Tu...   
4  Desperation in Black River, Jamaica, after Hur...   

                                             summary            published_at  \
0  There are also no plans to formally remove And... 2025-10-31 17:22:09.991   
1  The s

## API com dados de petróleo

In [31]:
# =========================
# Helpers
# =========================

def _retry_get(url: str, params: dict = None, max_tries: int = 5, sleep_base: float = 1.0):
    for i in range(max_tries):
        r = requests.get(url, params=params, timeout=30)
        if r.status_code == 200:
            return r
        time.sleep(sleep_base * (2**i))
    r.raise_for_status()

def _reindex_full_range(df: pd.DataFrame, start: dt.date, end: dt.date, date_col="date", value_cols=None):
    """Garante cobertura diária START→END com bfill+ffill."""
    if value_cols is None:
        value_cols = [c for c in df.columns if c != date_col]
    full = pd.DataFrame({"date": pd.date_range(start, end, freq="D").date})
    out = full.merge(df, on="date", how="left")
    # Corrige tipos numéricos
    for c in value_cols:
        out[c] = pd.to_numeric(out[c], errors="coerce")
        out[c] = out[c].bfill().ffill()  # preenche começo e meio
    return out

def fetch_fred_series_strict(series_id, start: dt.date, end: dt.date, api_key: str) -> pd.DataFrame:
    base = "https://api.stlouisfed.org/fred/series/observations"
    params = {
        "series_id": series_id,
        "api_key": api_key,
        "file_type": "json",
        "observation_start": start.isoformat(),
        "observation_end": end.isoformat(),
    }
    r = _retry_get(base, params=params)
    data = r.json().get("observations", [])
    df = pd.DataFrame(data)[["date", "value"]] if data else pd.DataFrame(columns=["date","value"])
    df["date"] = pd.to_datetime(df["date"]).dt.date
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    df = _reindex_full_range(df, start, end, value_cols=["value"])
    return df.rename(columns={"value": "close"})

def fetch_coingecko_btc_strict(start: dt.date, end: dt.date, vs_currency="usd") -> pd.DataFrame:
    base = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
    days = (end - start).days + 5
    r = _retry_get(base, params={"vs_currency": vs_currency, "days": days})
    js = r.json()
    p = pd.DataFrame(js.get("prices", []), columns=["ts_ms", "close"])
    if p.empty:
        p = pd.DataFrame(columns=["date", "close"])
    else:
        p["date"] = pd.to_datetime(p["ts_ms"], unit="ms").dt.date
        p = p.sort_values("ts_ms").groupby("date", as_index=False).tail(1)[["date","close"]]
        p = p[(p["date"] >= start) & (p["date"] <= end)]
    p = _reindex_full_range(p, start, end, value_cols=["close"])
    return p

def ensure_min_6_months(df, start, end, date_col="date"):
    if df.empty:
        raise AssertionError("DataFrame vazio.")
    span = (df[date_col].max() - df[date_col].min()).days
    if span < 180:
        raise AssertionError(f"Menos de 6 meses: {span} dias.")
    # no máximo 1% de buracos (após reindex + bfill/ffill deve ser 0)
    expected = set(pd.date_range(start, end, freq="D").date)
    got = set(df[date_col].values)
    missing = expected - got
    if len(missing) > len(expected) * 0.01:
        raise AssertionError(f"Muitas datas faltando ({len(missing)}).")

In [32]:
# =========================
# Execução
# =========================

if __name__ == "__main__":
    print(f"Coletando de {START} até {END} (~{(END-START).days} dias)")

    # ---- FRED: Brent & DEXUSUK ----
    brent = fetch_fred_series_strict("DCOILBRENTEU", START, END, FRED_API_KEY)  # Brent
    dex = fetch_fred_series_strict("DEXUSUK", START, END, FRED_API_KEY)         # USD/GBP
    gbpusd = dex.assign(close=lambda d: 1.0 / d["close"]).copy()                # GBP/USD

    # ---- CoinGecko: BTC/USD ----
    btc = fetch_coingecko_btc_strict(START, END, "usd")

    # ---- Validação ----
    for name, df in [("BRENT", brent), ("GBPUSD", gbpusd), ("BTCUSD", btc)]:
        ensure_min_6_months(df, START, END)
        print(f"[OK] {name}: {df['date'].min()} → {df['date'].max()} | {len(df)} linhas")

    # ---- Consolida para salvar ----
    prices = pd.concat(
        [
            brent.assign(instr="BRENT"),
            gbpusd.assign(instr="GBPUSD"),
            btc.assign(instr="BTCUSD"),
        ],
        ignore_index=True,
    ).sort_values(["instr", "date"])

    print(prices.groupby("instr").head(3))
    print(prices.groupby("instr").tail(3))

Coletando de 2025-04-05 até 2025-11-01 (~210 dias)
[OK] BRENT: 2025-04-05 → 2025-11-01 | 211 linhas
[OK] GBPUSD: 2025-04-05 → 2025-11-01 | 211 linhas
[OK] BTCUSD: 2025-04-05 → 2025-11-01 | 211 linhas
           date         close   instr
0    2025-04-05     66.130000   BRENT
1    2025-04-06     66.130000   BRENT
2    2025-04-07     66.130000   BRENT
422  2025-04-05  83852.007654  BTCUSD
423  2025-04-06  83595.885502  BTCUSD
424  2025-04-07  78211.483582  BTCUSD
211  2025-04-05      0.785608  GBPUSD
212  2025-04-06      0.785608  GBPUSD
213  2025-04-07      0.785608  GBPUSD
           date          close   instr
208  2025-10-30      65.520000   BRENT
209  2025-10-31      65.520000   BRENT
210  2025-11-01      65.520000   BRENT
630  2025-10-30  110046.669258  BTCUSD
631  2025-10-31  108240.765287  BTCUSD
632  2025-11-01  110031.819531  BTCUSD
419  2025-10-30       0.751993  GBPUSD
420  2025-10-31       0.751993  GBPUSD
421  2025-11-01       0.751993  GBPUSD


### Persistência

In [33]:
if isinstance(getattr(prices.index, "dtype", None), pd.PeriodDtype):
    prices = prices.copy()
    prices.index = prices.index.to_timestamp()           # para Timestamp
    prices = prices.reset_index().rename(columns={"index":"date"})

for c in prices.columns:
    if isinstance(prices[c].dtype, pd.PeriodDtype):
        prices[c] = prices[c].dt.to_timestamp()

prices["date"] = pd.to_datetime(prices["date"]).dt.date   # date puro
prices["instr"] = prices["instr"].astype(str)
for c in ["close","r1","r3","r5"]:
    if c in prices:
        prices[c] = pd.to_numeric(prices[c], errors="coerce").astype("float64")

In [34]:
# salva em parquet (opcional)
prices.to_parquet("prices.parquet", index=False)

# conecta ao banco local
con = duckdb.connect(DUCKDB_PATH)

# cria a tabela se não existir
con.execute("""
DROP TABLE IF EXISTS prices;
CREATE TABLE IF NOT EXISTS prices (
    date DATE,
    close DOUBLE,
    instr VARCHAR
);
""")

# registra o DataFrame do pandas como uma "view" temporária
con.register("tmp_prices", prices)

# insere os dados na tabela
con.execute("""
INSERT INTO prices
SELECT * FROM tmp_prices;
""")

# confirma e fecha
con.close()
print("✅ Dados salvos na tabela 'prices' do banco market.duckdb")

✅ Dados salvos na tabela 'prices' do banco market.duckdb


### Verificação

In [35]:
con = duckdb.connect(DUCKDB_PATH)

con.execute("select instr, min(date) as min_date, max(date) as max_date, count(*) as n_rows from prices group by instr").df()

,instr,min_date,max_date,n_rows
0,GBPUSD,2025-04-05,2025-11-01,211
1,BTCUSD,2025-04-05,2025-11-01,211
2,BRENT,2025-04-05,2025-11-01,211


## Tabela intrumentos

In [36]:
# --- 3ª tabela: instruments ---
con.execute("""
CREATE TABLE IF NOT EXISTS instruments (
    instr_id VARCHAR PRIMARY KEY,
    symbol   VARCHAR,
    name     VARCHAR,
    class    VARCHAR
);
""")
con.register("tmp_instr", pd.DataFrame([
    {"instr_id":"BRENT",  "symbol":"DCOILBRENTEU",      "name":"Brent (FRED)",                  "class":"commodity"},
    {"instr_id":"GBPUSD", "symbol":"DEXUSUK (inverted)","name":"GBP/USD (from DEXUSUK – FRED)", "class":"fx"},
    {"instr_id":"BTCUSD", "symbol":"CoinGecko BTC/USD", "name":"Bitcoin (CoinGecko)",           "class":"crypto"},
]))
con.execute("DELETE FROM instruments WHERE instr_id IN (SELECT instr_id FROM tmp_instr)")
con.execute("INSERT INTO instruments SELECT * FROM tmp_instr")

print(con.execute("SELECT * FROM instruments").df())
con.close()

  instr_id              symbol                           name      class
0    BRENT        DCOILBRENTEU                   Brent (FRED)  commodity
1   GBPUSD  DEXUSUK (inverted)  GBP/USD (from DEXUSUK – FRED)         fx
2   BTCUSD   CoinGecko BTC/USD            Bitcoin (CoinGecko)     crypto


## Banco

In [37]:
con = duckdb.connect(DUCKDB_PATH)

# lista todas as tabelas
print(con.execute("SHOW TABLES").df())

# mostra o esquema completo (colunas e tipos)
print(con.execute("DESCRIBE prices").df())
print(con.execute("DESCRIBE news_bbc").df())
print(con.execute("DESCRIBE instruments").df())

con.close()

          name
0  instruments
1     news_bbc
2       prices
  column_name column_type null   key default extra
0        date        DATE  YES  None    None  None
1       close      DOUBLE  YES  None    None  None
2       instr     VARCHAR  YES  None    None  None
      column_name column_type null   key default extra
0             url     VARCHAR  YES  None    None  None
1    collected_at   TIMESTAMP  YES  None    None  None
2           title     VARCHAR  YES  None    None  None
3         summary     VARCHAR  YES  None    None  None
4    published_at   TIMESTAMP  YES  None    None  None
5  published_text     VARCHAR  YES  None    None  None
  column_name column_type null   key default extra
0    instr_id     VARCHAR   NO   PRI    None  None
1      symbol     VARCHAR  YES  None    None  None
2        name     VARCHAR  YES  None    None  None
3       class     VARCHAR  YES  None    None  None
